In [1]:
import numpy as np
import timeit
from tqdm.auto import tqdm

def check_correctness(eigvals_func, A, rtol=1e-2, atol=1e-3):
    assert np.allclose(sorted(np.linalg.eigvals(A)), sorted(eigvals_func(A)), rtol=rtol, atol=atol)

def make_experiment(eigvals_func, N=50, random_seed=0, correctness_tests=100, rtol=1e-2, atol=1e-3, estimated_time=10, retries=7):
    np.random.seed(random_seed)
    for _ in tqdm(range(correctness_tests), desc="Checking correctness"):
        A = np.random.randn(N, N)
        A = A.dot(A.T)
        check_correctness(eigvals_func, A, rtol=rtol, atol=atol)
    
    variables = {'eigvals_func': eigvals_func, 'A': A}
    single_execution = timeit.timeit(stmt="eigvals_func(A)", globals=variables, number=1)
    number = max(int(np.floor(estimated_time / single_execution / retries)), 1)
    times = [
        timeit.timeit(stmt="eigvals_func(A)", globals=variables, number=number) / number for _ in range(retries)
    ]
    return times

make_experiment(np.linalg.eigvals)

Checking correctness:   0%|          | 0/100 [00:00<?, ?it/s]

[0.0004223980633802817,
 0.0004065346244131457,
 0.00040288679577464787,
 0.0004134767899061032,
 0.0004150199530516434,
 0.0004050781396713616,
 0.0004089346537558685]

In [2]:
def qr(A):
    Q = A.copy()
    Q[:, 0] /= np.linalg.norm(Q[:, 0])
    for i in range(1, A.shape[0]):
        Q[:, i] -= Q[:, :i].dot(Q[:, :i].T).dot(Q[:, i])
        Q[:, i] /= np.linalg.norm(Q[:, i])
    R = Q.T.dot(A)
    return Q, R

def eigenvalues(A, rtol=1e-6, atol=1e-8):
    squared_A = np.square(A)
    while squared_A.sum() - np.diag(squared_A).sum() >= rtol * squared_A.sum() + atol:
        Q, R = qr(A)
        A = R.dot(Q)
        squared_A = np.square(A)
    return np.diag(A)

make_experiment(eigenvalues)

Checking correctness:   0%|          | 0/100 [00:00<?, ?it/s]

[0.21852473333333342,
 0.22006126666666623,
 0.22162841666666586,
 0.22241868333333295,
 0.21671138333333317,
 0.2176305333333334,
 0.21398433333333364]

In [3]:
def make_iteration(A):
    Q = A.copy()
    Q[:, 0] /= np.linalg.norm(Q[:, 0])
    for i in range(1, A.shape[0]):
        Q[:, i] -= Q[:, :i].dot(Q[:, :i].T).dot(Q[:, i])
        Q[:, i] /= np.linalg.norm(Q[:, i])
    return Q.T.dot(A).dot(Q)

def eigenvalues(A, rtol=1e-6, atol=1e-8):
    squared_A = np.square(A)
    while squared_A.sum() - np.diag(squared_A).sum() >= rtol * squared_A.sum() + atol:
        A = make_iteration(A)
        squared_A = np.square(A)
    return np.diag(A)

make_experiment(eigenvalues)

Checking correctness:   0%|          | 0/100 [00:00<?, ?it/s]

[0.21737011666666461,
 0.21469396666666776,
 0.21290133333333236,
 0.21573575000000034,
 0.21377313333333348,
 0.21377425000000017,
 0.20421029999999973]

In [4]:
def make_iteration(A):
    Q = A.T.copy()
    saved_Q = np.zeros_like(A)
    for i in range(A.shape[0]):
        Q[i] -= saved_Q.dot(Q[i])
        Q[i] /= np.linalg.norm(Q[i])
        saved_Q += np.outer(Q[i], Q[i])
    return np.linalg.multi_dot([Q, A, Q.T])

def eigenvalues(A, rtol=1e-6, atol=1e-8):
    squared_A = np.square(A)
    while squared_A.sum() - np.diag(squared_A).sum() >= rtol * squared_A.sum() + atol:
        A = make_iteration(A)
        squared_A = np.square(A)
    return np.diag(A)

make_experiment(eigenvalues)

Checking correctness:   0%|          | 0/100 [00:00<?, ?it/s]

[0.17870024285714287,
 0.1765434571428557,
 0.17609327142857215,
 0.17632547142857138,
 0.17524777142857115,
 0.1761459000000006,
 0.1728164142857135]

In [5]:
from numba import jit

@jit(nopython=True)
def make_iteration(A):
    Q = A.T.copy()
    saved_Q = np.zeros_like(A)
    for i in range(A.shape[0]):
        Q[i] -= saved_Q.dot(Q[i])
        Q[i] /= np.linalg.norm(Q[i])
        saved_Q += np.outer(Q[i], Q[i])
    return Q.dot(A).dot(Q.T)

@jit(nopython=True)
def eigenvalues(A, rtol=1e-6, atol=1e-8):
    squared_A = np.square(A)
    while squared_A.sum() - np.diag(squared_A).sum() >= rtol * squared_A.sum() + atol:
        A = make_iteration(A)
        squared_A = np.square(A)
    return np.diag(A)

make_experiment(eigenvalues)

Checking correctness:   0%|          | 0/100 [00:00<?, ?it/s]

[0.05617848399999957,
 0.0566878760000003,
 0.05657852000000048,
 0.05682432399999982,
 0.056436883999999736,
 0.056811247999999065,
 0.05675859599999967]

In [6]:
from numba import jit

@jit(nopython=True, nogil=True, fastmath=True, cache=True)
def make_iteration(A):
    Q = A.T.copy()
    saved_Q = np.zeros_like(A)
    for i in range(A.shape[0]):
        Q[i] -= saved_Q.dot(Q[i])
        Q[i] /= np.linalg.norm(Q[i])
        saved_Q += np.outer(Q[i], Q[i])
    return Q.dot(A).dot(Q.T)

@jit(nopython=True, nogil=True, fastmath=True, cache=True)
def eigenvalues(A, rtol=1e-6, atol=1e-8):
    squared_A = np.square(A)
    while squared_A.sum() - np.diag(squared_A).sum() >= rtol * squared_A.sum() + atol:
        A = make_iteration(A)
        squared_A = np.square(A)
    return np.diag(A)

make_experiment(eigenvalues)

Checking correctness:   0%|          | 0/100 [00:00<?, ?it/s]

[0.05793499565217407,
 0.06271596956521633,
 0.05809121304347944,
 0.066521734782609,
 0.06358311304347808,
 0.05809486956521774,
 0.056654943478261915]

In [7]:
from numba import guvectorize, float64

@guvectorize(["void(float64[:, ::1], float64, float64, float64[::1])"], "(n,n), (), () -> (n)", nopython=True, target='cpu')
def eigenvalues(A, rtol=1e-6, atol=1e-8, out=None):
    squared_A = A * A
    while squared_A.sum() - np.diag(squared_A).sum() >= rtol * squared_A.sum() + atol:
        Q = A.T.copy()
        Q[0] /= np.linalg.norm(Q[0])
        saved_Q = np.zeros_like(A)
        for i in range(A.shape[0]):
            Q[i] -= saved_Q.dot(Q[i])
            Q[i] /= np.linalg.norm(Q[i])
            saved_Q += np.outer(Q[i], Q[i])
        A = Q.dot(A).dot(Q.T)
        
        squared_A = A * A
    out[:] = np.diag(A)

def eigenvalues_wrapper(A, rtol=1e-6, atol=1e-8):
    result = np.empty(A.shape[0], dtype=float)
    eigenvalues(A, rtol, atol, result)
    return result

make_experiment(eigenvalues_wrapper)

Checking correctness:   0%|          | 0/100 [00:00<?, ?it/s]

[0.058021595833333585,
 0.05779220833333435,
 0.05694217916666607,
 0.057847241666666584,
 0.056839724999999675,
 0.05663400000000015,
 0.05702203333333363]